# Data Structure: Trie (python)
### [Glitched Failure Video]()

## How does a `Trie` (my favorite data structure!) work?

A trie is a __graph data structure__ built with nodes that point to other nodes.
- This can be used to represent a large corpus (many words) in a small space. In our case, features will include:
    - Root node: manages start of a word
    - Each node represents a letter, __except for the root node__
    - And node flagged as an "end" serves as a signal that the current letter could be the last letter of a word. _This is where the Trie is efficient!_. Shorter words "fit" inside of larger ones:
        - For example, the word "sorting" shares the same nodes for the word "sort"
    - Each node contains a set of children nodes, all of the possible characters that could come next.

<img src='./images/Trie.png' width='200px'>

### Tests:

In [ ]:
# [TEST #1] Can add words and check if word is in the Trie
trie = Trie()
trie.add_word("hello")

assert "hello" in trie
assert "nothing" not in trie
print("Test passed!")

In [ ]:
# [TEST #2] Has a count
trie = Trie()

trie.add_word("hello")
assert trie.count == 1

trie.add_word("testing")
assert trie.count == 2

# adding same word should NOT increase count
trie.add_word("testing")
assert trie.count == 2

print("Test passed!")

## Base `Node` Class

In [2]:
class Node(): 
    def __init__(self, char):
        self.children = dict()
        self.char = char
        self.is_end = False

In [11]:
class Trie:
    def __init__(self):
        self.count = 0
        self.root = Node(None)

    def __repr__(self):
        return f"Trie: {self.count} word{'' if self.count == 1 else 's'}"
    
    def add_word(self, word):
        curr_node = self.root
        
        for char in word:
            curr_node = self._find_or_create_child_node(curr_node, char)

        if not curr_node.is_end: # duplicates won't add to count
            curr_node.is_end = True
            self.count += 1

    def _find_or_create_child_node(self, node, char):
        if char in node.children:
            return node.children.get(char)
        child_node = Node(char)
        node.children[char] = child_node
        return child_node

    def __contains__(self, word):
        curr_node = self.root

        for char in word:
            if char not in curr_node.children:
                return False
            curr_node = curr_node.children.get(char)

        return curr_node.is_end

In [14]:
trie = Trie()

In [16]:
trie.add_word("Hello")

In [18]:
"Hello" in trie

True

### Tests:

In [19]:
# [TEST #1] Can add words and check if word is in the Trie
trie = Trie()
trie.add_word("hello")

assert "hello" in trie
assert "nothing" not in trie
print("Test passed!")

Test passed!


In [20]:
# [TEST #2] Has a count
trie = Trie()

trie.add_word("hello")
assert trie.count == 1

trie.add_word("testing")
assert trie.count == 2

# adding same word should NOT increase count
trie.add_word("testing")
assert trie.count == 2

print("Test passed!")

Test passed!


## Quick dictionary lookup
World list [source](https://github.com/dwyl/english-words/blob/master/words_alpha.txt)


In [32]:
file = "words_alpha.txt"

trie = Trie()
with open(file, 'r') as f:
    for raw_word in f.readlines():
        word = raw_word.strip().lower()
        trie.add_word(word)

In [33]:
trie.count

370103

In [34]:
'hello' in trie

True

In [35]:
"asldkjasdf" in trie

False

## Use cases
- The `Trie` data structure is optimized for lookup
- Quick searches: "Are these string of characters in the collection?"
    - More efficient that a `Set`, which would store each word individually
- Quick partial searches: "User starting typing the beginning of a word and we want a list of possible endings"
    - This would ignore the 'end' of a word and gather all words after end of partial string
- Deduplicating (similar to a `Set`)
- Serve as an efficient `Dict` (key:value pairs)
    - String of char would be key
    - Would need to add functionality to store the value

-----

# `WordSearchTrie`
### [Scenario] We have an book application and we want to allow users to:
- Be able to quickly search for words in a large text
- Know how often that word shows up

### Modifications to `Trie`
- We'll need to store the location (index) for each word on the "end" node (as a list)
- The "count" will just be the length of this list

### Tests

In [ ]:
# [TEST #1]
# Can add words and check if word is in the Trie, returning indices for each word
ws_trie = WordSearchTrie()

ws_trie.add_word('one', 0)
ws_trie.add_word('two', 4)
ws_trie.add_word('three', 8)

assert ws_trie['one'] == [0] # indices of "one"
assert ws_trie['nothing'] == [] # indices of "nothing"
print("Test passed!")

In [ ]:
# [TEST #2] Duplicates are handled as expected
ws_trie = WordSearchTrie()

ws_trie.add_word('one', 0)
ws_trie.add_word('two', 4)
ws_trie.add_word('one', 8)

assert ws_trie['one'] == [0, 8] # indices of "one"
print("Test passed!")

# `WordSearchNode`

In [36]:
class WordSearchNode(): 
    def __init__(self, char):
        self.children = dict()
        self.char = char
        self.is_end = False
        self.indices = [] # for end node, where in text can the word be found

# `WordSearchTrie`

In [37]:
class WordSearchTrie:
    def __init__(self):
        self.count = 0 # count of all unique words (not required by spec, but nice to have)
        self.root = WordSearchNode(None)

    def __repr__(self):
        return f"WordSearchTrie: {self.count} word{'' if self.count == 1 else 's'}"
    
    def add_word(self, word, index):
        curr_node = self.root
        
        for char in word:
            curr_node = self._find_or_create_child_node(curr_node, char)

        curr_node.indices.append(index)
        if not curr_node.is_end: # duplicates won't add to count (on Trie)
            curr_node.is_end = True
            self.count += 1

    def _find_or_create_child_node(self, node, char):
        if char in node.children:
            return node.children.get(char)
        child_node = WordSearchNode(char)
        node.children[char] = child_node
        return child_node

    def __getitem__(self, word):
        curr_node = self.root
        
        for char in word:
            if char not in curr_node.children:
                return [] # word not in Trie
            curr_node = curr_node.children.get(char)

        return curr_node.indices[:] # return copy of indices array 

In [38]:
ws_trie = WordSearchTrie()

In [39]:
ws_trie.add_word("hallo", 999)

In [45]:
l = ws_trie["hallo"]
l.append(88)

In [46]:
l

[999, 88]

In [48]:
ws_trie["hallo"]

[999]

### Tests

In [49]:
# [TEST #1] Can add words and check if word is in the Trie, returning word count and indices for each word
ws_trie = WordSearchTrie()

ws_trie.add_word('one', 0)
ws_trie.add_word('two', 4)
ws_trie.add_word('three', 8)

assert ws_trie['one'] == [0] # indices of "one"
assert ws_trie['nothing'] == [] # indices of "nothing"
print("Test passed!")

Test passed!


In [50]:
# [TEST #2] Duplicates are handled as expected
ws_trie = WordSearchTrie()

ws_trie.add_word('one', 0)
ws_trie.add_word('two', 4)
ws_trie.add_word('one', 8)

assert ws_trie['one'] == [0, 8] # indices of "one"
print("Test passed!")

Test passed!


### Alice's Adventures in Wonderland by Lewis Carroll
Source: https://www.gutenberg.org/ebooks/11

In [51]:
import re

def process_word(word):
    """Removes non-letter characters"""
    return re.sub(r'[^a-z]', "", word.strip().lower())

In [52]:
process_word("EFGabcd 123 @#$&")

'efgabcd'

#### Loading book example

In [53]:
# EXAMPLE WITH FIRST 1000 CHARACTERS FROM BOOK
file = "alice_in_wonderland.txt"

with open(file,'r',encoding = 'utf-8') as f:
    full_text = f.read()[:1000]

processed_words = []
STOP_CHAR = (' ', '\n')
max_n = len(full_text)

index_cursor = 0
fast_cursor = 0
while fast_cursor < max_n:
    while fast_cursor < max_n and full_text[fast_cursor] not in STOP_CHAR:
        fast_cursor += 1

    word = full_text[index_cursor:fast_cursor]
    processed_word = process_word(word)
    if processed_word:
        processed_words.append((processed_word, index_cursor))
    fast_cursor += 1
    index_cursor = fast_cursor
processed_words

[('the', 0),
 ('project', 5),
 ('gutenberg', 13),
 ('ebook', 23),
 ('of', 29),
 ('alices', 32),
 ('adventures', 40),
 ('in', 51),
 ('wonderland', 54),
 ('by', 66),
 ('lewis', 69),
 ('carroll', 75),
 ('this', 84),
 ('ebook', 89),
 ('is', 95),
 ('for', 98),
 ('the', 102),
 ('use', 106),
 ('of', 110),
 ('anyone', 113),
 ('anywhere', 120),
 ('in', 129),
 ('the', 132),
 ('united', 136),
 ('states', 143),
 ('and', 150),
 ('most', 154),
 ('other', 159),
 ('parts', 165),
 ('of', 171),
 ('the', 174),
 ('world', 178),
 ('at', 184),
 ('no', 187),
 ('cost', 190),
 ('and', 195),
 ('with', 199),
 ('almost', 204),
 ('no', 211),
 ('restrictions', 214),
 ('whatsoever', 227),
 ('you', 239),
 ('may', 243),
 ('copy', 247),
 ('it', 252),
 ('give', 256),
 ('it', 261),
 ('away', 264),
 ('or', 269),
 ('reuse', 272),
 ('it', 279),
 ('under', 282),
 ('the', 288),
 ('terms', 292),
 ('of', 298),
 ('the', 301),
 ('project', 305),
 ('gutenberg', 313),
 ('license', 323),
 ('included', 331),
 ('with', 340),
 ('this',

#### Loading book into `WordSearchTrie`

In [55]:
file = "alice_in_wonderland.txt"

ws_trie = WordSearchTrie()

with open(file,'r',encoding = 'utf-8') as f:
    full_text = f.read()

STOP_CHAR = (' ', '\n')
max_n = len(full_text)

index_cursor = 0
fast_cursor = 0
while fast_cursor < max_n:
    while fast_cursor < max_n and full_text[fast_cursor] not in STOP_CHAR:
        fast_cursor += 1

    word = full_text[index_cursor:fast_cursor]
    processed_word = process_word(word)
    if processed_word:
        ws_trie.add_word(processed_word, index_cursor)
    fast_cursor += 1
    index_cursor = fast_cursor

In [56]:
ws_trie['glass']

[7371, 11119, 11971, 18898, 33864, 39126, 39531, 40191, 85233]

In [57]:
len(ws_trie['glass'])

9

### User experience

In [58]:
def print_highlight_index(index, word_length, full_text):
    # https://stackoverflow.com/questions/16816013/is-it-possible-to-print-using-different-colors-in-ipythons-notebook
    RED = "\x1b[31m"
    NO_COLOR = "\x1b[0m"
    WINDOW = 30
    min_index = max(index - WINDOW, 0)
    max_index = min(index + WINDOW, len(full_text))
    left_text = full_text[min_index:index]
    index_text = full_text[index:index+word_length]
    right_text = full_text[index+word_length: max_index]

    string = f"{left_text}{RED}{index_text}{NO_COLOR}{right_text}".replace("\n",' ')
    print(string)

In [59]:
print_highlight_index(4,3,"something")

something


In [60]:
word = 'queen'
for index in ws_trie[word]:
    print_highlight_index(index,len(word), full_text)

chess. An invitation from the Queen to play croquet.” The Fr
the words a little, “From the Queen. An invitation for the D
eady to play croquet with the Queen,” and she hurried out of
“Do you play croquet with the Queen to-day?”  “I should like
he great concert given by the Queen of Hearts, and I had to 
,” said the Hatter, “when the Queen jumped up and bawled out
alk!” said Five. “I heard the Queen say only yesterday you d
one in by mistake; and if the Queen was to find it out, we s
s the garden, called out “The Queen! The Queen!” and the thr
n, called out “The Queen! The Queen!” and the three gardener
ooked round, eager to see the Queen.  First came ten soldier
procession, came THE KING AND QUEEN OF HEARTS.  Alice was ra
ed and looked at her, and the Queen said severely “Who is th
 in reply.  “Idiot!” said the Queen, tossing her head impati
nd who are _these?_” said the Queen, pointing to the three g
 no business of _mine_.”  The Queen turned crimson with fury
loudly and decidedly, an